In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as signal
import skimage.filters as filters

from PIL import Image
#from pyneurotrace import filters  as pntfilters
from scipy import integrate

<H3> Global Variables

In [8]:
# The Folder containing sub-directories to include in analysis
DATA_DIRECTORY = "/home/islandhead/Documents/DBC_Tutoring/Wissam/osfstorage-archive/Data (Images)/"
# Frequency frames were collected
HZ = 10

In [9]:
"""
Jia, H., Rochefort, N. L., Chen, X., & Konnerth, A. (2011).
In vivo two-photon imaging of sensory-evoked dendritic calcium signals in cortical neurons.
Nature protocols, 6(1), 28.
"""
"""
Podgorski, K., & Haas, K. (2013).
Fast non‐negative temporal deconvolution for laser scanning microscopy.
Journal of biophotonics, 6(2), 153-162.
"""
"""
Coleman, P. (2019).
pyNeuroTrace. https://github.com/padster/pyNeuroTrace.git
"""
# To install pyneuortrace use this:
# pip install --upgrade "git+https://github.com/padster/pyNeuroTrace#egg=pyneurotrace&subdirectory=pyneurotrace"

'\nColeman, P. (2019).\npyNeuroTrace. https://github.com/padster/pyNeuroTrace.git\n'

In [10]:
# Change output figure size
# ...needs to be in its own cell for some reason...

plt.rcParams['figure.figsize'] = [20, 5]

In [27]:
"""
Performs fast nonnegative deconvolution on pmt signal to solve for minimum MSE photon rate
   trace  :   The data to be deconvolved
   tau    :   The time constant of the PMT, in data samples
   return :   estimated photon rate
A matlab version is also available on request.
For details on how this works, see:
  Podgorski, K., & Haas, K. (2013).
  Fast non‐negative temporal deconvolution for laser scanning microscopy.
  Journal of biophotonics, 6(2), 153-162.
"""
def nonNegativeDeconvolution(trace, tau):
    T = len(trace)
    counts = np.zeros(T)
    counts[-1] = trace[-1]
    cutoff = math.ceil(8 * tau)
    kernel = np.exp(-np.arange(cutoff + 1)/tau) # convolution kernel
    recent = np.full(1 + round(T / 2), np.nan).astype(int)
    recent[0] = T #stored locations where we assigned counts
    recentIdx = 0

    # the points that could potentially be assigned counts:
    _delayed = np.concatenate(([0], trace[:-2]))
    points = (trace[:-1] > kernel[1] * _delayed) & (trace[:-1] > 0)

    # dividing these points up into runs, for speed
    runStarts = np.where(points & ~(np.concatenate(([False], points[:-1]))))[0].astype(int)
    runEnds = np.where(points & ~(np.concatenate((points[1:], [False]))))[0].astype(int)
    runIdx = len(runEnds) - 1

    while runIdx >= 0:
        oldTop, oldBottom = 0, 0
        t = runEnds[runIdx]
        t1 = t
        accum = 0

        converged = False
        while not converged:
            if recentIdx >= 0 and recent[recentIdx] < (t+cutoff):
                t2 = recent[recentIdx] - 1
                C_max = counts[t2] / kernel[t2-t]
            else:
                t2 = min(t + cutoff, T+1) - 1
                C_max = np.inf


            b = kernel[t1-t:t2-t]
            top = np.dot(b, trace[t1:t2]) + oldTop #this is the numerator of the least squares fit for an exponential
            bottom = np.dot(b, b) + oldBottom #this is the denominator of the fit

            done = False
            while not done:
                #the error function is (data-kernel.*C)^2
                bestC = max(top/bottom, 0);  #C=top/bottom sets the derivative of the error to 0

                # does not meet nonnegative constraint. Continue to adjust previous solutions.
                if bestC > (C_max+accum):
                    accum = accum + counts[t2] / kernel[t2-t]
                    counts[t2] = 0
                    t1 = t2
                    oldTop = top
                    oldBottom = bottom
                    recentIdx -= 1
                    done = True

                else: # converged!
                    #now that we have found the MSE counts for times t<end, check if
                    #this will be swamped by the next timepoint in the run
                    if  (t == runStarts[runIdx]) or (trace[t-1] < bestC/kernel[1]): #%C_max won't necessarily get swamped
                        if recentIdx >= 0 and t2 <= t + cutoff:
                            counts[t2] = counts[t2] - (bestC - accum) * kernel[t2-t]
                        runStart = runStarts[runIdx]
                        initIdx = recentIdx + 1
                        recentIdx = recentIdx + 1 + t - runStart;

                        _skipped = 0
                        if recentIdx + 1 > len(recent):
                            _skipped = recentIdx - (len(recent) - 1)
                            recentIdx = len(recent) - 1


                        recent[initIdx:recentIdx + 1] = np.arange(t+1, runStart + _skipped, -1)
                        counts[runStart:(t+1)] = \
                               np.concatenate((trace[runStart:t], [bestC])) - \
                               np.concatenate(([0], kernel[1]*trace[runStart:t]))
                        done = True
                        converged = True
                    else: #%C_max will get swamped
                        #%in this situation, we know that this point will be removed
                        #%as we continue to process the run. To save time:
                        t -= 1
                        runEnds[runIdx] = t
                        accum = accum / kernel[1]
                        top = top * kernel[1] + trace[t] #% %this is the correct adjustment to the derivative term above
                        bottom = bottom * (kernel[1] ** 2) + 1 #% %this is the correct adjustment to the derivative term above

        runIdx -= 1
    return counts

In [23]:
def nndSmooth(data, hz, tau, iterFunc=None):
    tauSamples = tau * hz

    # This is the transient shape we're deconvolving against:
    # e^(x/tauSamples), for 8 times the length of tau.
    cutoff = round(8 * tauSamples)
    fitted = np.exp(-np.arange(cutoff + 1) / tauSamples)

    def _singleRowNND(samples):
        result = np.copy(samples)
        nanSamples = np.isnan(samples)
        if np.all(nanSamples):
            pass # No data
        elif not np.any(nanSamples):
            # All samples exist, so fit in one go
            result = np.convolve(nonNegativeDeconvolution(samples, tauSamples), fitted)[:len(samples)]
        else:
            # Lots of different runs of samples, fit each separately
            starts = np.where((not nanSamples) & np.isnan(np.concatenate(([1], samples[:-1]))))[0]
            ends = np.where((not nanSamples) & np.isnan(np.concatenate((samples[1:], [1]))))[0]
            for start, end in zip(starts, ends):
                tmp = np.convolve(NND(samples[start:end], tauSamples), fitted)
                result[start:end] = np.max(0, tmp[:end - start + 1])
        return result

    return _forEachTimeseries(data, _singleRowNND, iterFunc)

def deltaFOverF0(data, hz, t0=0.2, t1=0.75, t2=3.0, iterFunc=None):
    t0ratio = None if t0 is None else np.exp(-1 / (t0 * hz))
    t1samples, t2samples = round(t1 * hz), round(t2*hz)

    def _singeRowDeltaFOverF(samples):
        fBar = _windowFunc(np.mean, samples, t1samples, mid=True)
        f0 = _windowFunc(np.min, fBar, t2samples)
        result = (samples - f0) / f0
        if t0ratio is not None:
            result = _ewma(result, t0ratio)
        return result
    return _forEachTimeseries(data, _singeRowDeltaFOverF, iterFunc)


def _windowFunc(f, x, window, mid=False):
    n = len(x)
    startOffset = (window - 1) // 2 if mid else window - 1
    result = np.zeros(x.shape)
    for i in range(n):
        startIdx = i - startOffset
        endIdx = startIdx + window
        startIdx, endIdx = max(0, startIdx), min(endIdx, n)
        result[i] = f(x[startIdx:endIdx])
    return result


def _ewma(x, ratio):
    result = np.zeros(x.shape)
    weightedSum, sumOfWeights = 0.0, 0.0
    for i in range(len(x)):
        weightedSum = ratio * weightedSum + x[i]
        sumOfWeights = ratio * sumOfWeights + 1.0
        result[i] = weightedSum / sumOfWeights
    return result

# Input is either 1d (timeseries), 2d (each row is a timeseries) or 3d (x, y, timeseries)
def _forEachTimeseries(data, func, iterFunc=None):
    if iterFunc is None:
        iterFunc = lambda x: x
    dim = len(data.shape)
    result = np.zeros(data.shape)
    if dim == 1: # single timeseries
        result = func(data)
    elif dim == 2: # (node, timeseries)
        for i in iterFunc(range(data.shape[0])):
            result[i] = func(data[i])
    elif dim == 3: # (x, y, timeseries)
        for i in iterFunc(range(data.shape[0])):
            for j in iterFunc(range(data.shape[1])):
                result[i, j] = func(data[i, j])
    return result

In [24]:
# Take a folder of Tifs and turn it into a numpy array
def folder2tif(dir_path):
    final = []
    files = os.listdir(dir_path)
    files = sorted(files)
    movie = []
    for fname in files:
        im = Image.open(os.path.join(dir_path, fname))
        imarray = np.array(im)
        movie.append(imarray)
    movie = np.asarray(movie)
    return movie

# Returns change in fluorescence over average fluorescence of ROI
def deltaF(video_mask):
    video_mask_nan = video_mask.copy()
    video_mask_nan[video_mask_nan==0] = np.nan
    mean = np.mean(np.nanmean(video_mask))
    print(mean)
    dff= np.zeros((video_mask.shape[0]))
    for i in range(dff.shape[0]):
        delta = np.nanmean(video_mask[i, :, :])-mean
        dff[i] = delta/mean
    return dff

# Returns raw values fluorescence in the ROI
def rawIntensity(video_mask):
    video_mask_nan = video_mask.copy()
    video_mask_nan[video_mask_nan==0] = np.nan
    mean = np.nanmean(video_mask, axis=(1,2))
    return mean

# Generates the ROI 
def genROI(gcamp, rcamp):

    # To create a ROI for the nucleus a STD projection is created
    # Thresholding this image creates a mask for the roi
    std_projectionG = np.std(gcamp, axis=0)
    threshold = filters.threshold_otsu(std_projectionG)
    std_projectionG[std_projectionG < threshold] = 0
    std_projectionG[std_projectionG>0]=1

    # Create a ROI for the cytosl using an STD projection 
    # Thresholding this image creates a mask for the roi
    std_projectionR = np.std(rcamp, axis=0)
    threshold = filters.threshold_otsu(std_projectionR)
    std_projectionR[std_projectionR < threshold] = 0
    std_projectionR[std_projectionR>0]=1
    
    # Remove the Nuclear Mask from this ROI
    std_projectionR[std_projectionG==1]=0
    
    # Applying the masks for the two channels
    gcamp_masked = gcamp * std_projectionG
    rcamp_masked = rcamp * std_projectionR
    
    return gcamp_masked, rcamp_masked


# Generate a ROI for a cell
# Input of data directory, cell, Returns RCaMP and GCamP Raw Inrensity
def Cell2Trace(path, cell):
    # Import the movies provide the path to the folder containing the frames
    gcamp = folder2tif(path+cell+"_G/")
    rcamp = folder2tif(path+cell+"_R/")
    
    # Return Masked Arrays
    gcamp_masked, rcamp_masked = genROI(gcamp, rcamp)
    
    # Return Raw Traces from ROI
    gcamp = rawIntensity(gcamp_masked)
    rcamp = rawIntensity(rcamp_masked)
    
    return gcamp, rcamp

def peakDetect(trace):
    # Detect Peaks
    threshold =np.std(trace)
    peaks, _ = signal.find_peaks(trace, width=7, rel_height=.5, prominence=(.1*threshold))

    width = signal.peak_widths(trace, peaks, rel_height=.1)
    
    return peaks, width[0]

def signal_analysis(cell_id, gcamp, rcamp):
    gcamp_peaks, gcamp_widths = peakDetect(gcamp)
    rcamp_peaks, rcamp_widths = peakDetect(rcamp)

    
    # Match Peaks and puttin them in a list of tuples (g, r)
    gcamp_matched= []
    for g in gcamp_peaks:
        for r in rcamp_peaks:
            a = math.isclose(g, r, abs_tol=15)
            if a == True:
                gcamp_matched.append((g,r))
                
    # Assumption: Last shared peak is due to drug application
    # ***Improve this later***
    drug_app = np.mean(gcamp_matched[-1])
    cutoffG = np.where(gcamp_peaks == gcamp_matched[-1][0])[0][0]
    cutoffR = np.where(rcamp_peaks == gcamp_matched[-1][1])[0][0]
    gcamp_matched = gcamp_matched[:-1]


    # Apply the cutoff 
    gcamp_peaks = gcamp_peaks[:cutoffG]
    gcamp_widths = gcamp_widths[:cutoffG]
    rcamp_peaks = rcamp_peaks[:cutoffR]
    rcamp_widths = rcamp_widths[:cutoffR]
    
    
    # General Stats for the cell
    cell_stats = { 
                                'Cell ID': cell_id,
                                'GCaMP Peaks':len(gcamp_peaks),
                                'RCaMP Peaks':len(rcamp_peaks),
                                'Shared Peaks':len(gcamp_matched),
                                'GCaMP Percent Shared':(len(gcamp_matched)/len(gcamp_peaks)),
                                'RCaMP Percent Shared':(len(gcamp_matched)/len(gcamp_peaks)),
                                'Experiment Length (s)': gcamp.shape[0]/10,
                                'Predicted Drug Application': drug_app/10,
                               }
    
    cell_stats = pd.DataFrame(data=cell_stats, index=[0])
    peak_data = pd.DataFrame()
    for event in gcamp_matched:
        
        gindex = np.where(gcamp_peaks == event[0])[0]
        rindex = np.where(rcamp_peaks == event[1])[0]
        
        
        # Integrate Under the Curve for Area 
        # Note: Area from start to peak
        g_event_start = int(event[0]-gcamp_widths[gindex])
        if g_event_start < 0: 
            g_event_start = 0
        r_event_start = int(event[0]-rcamp_widths[rindex])
        if r_event_start < 0:
            r_event_start = 0

        g_area = integrate.cumtrapz(gcamp[g_event_start:event[0]])
        r_area = integrate.cumtrapz(rcamp[r_event_start:event[1]])
        
        peak_stats = {          'Cell ID': cell_id,            
                                'GCaMP Loc':event[0],
                                'GCaMP Width':gcamp_widths[gindex],
                                'GCaMP Prominence':gcamp[event[0]],
                                'GCaMP Area':g_area,                                
                                'RCaMP Loc':event[1],
                                'RCaMP Width':rcamp_widths[rindex],
                                'RCaMP Prominence':rcamp[event[1]],
                                'RCaMP Area':r_area,                                
                                'Promicence Ratio (G/R)':(gcamp[event[1]]/rcamp[event[0]]),
                                'Peak Time Diff (G-r)':((event[0]-event[1])*100),
                                                                 }
        peak_data = peak_data.append(peak_stats, ignore_index=True)
    return cell_stats, peak_data
    
    
    

In [25]:
# Collects all the cells in the analysis data directory and groups
# Them by condition in two lists
path = os.fspath(DATA_DIRECTORY)
cells = sorted((os.listdir(path)))
WT_Cells = []
YAC128_Cells = []
for folder in cells:
    print(folder)
    if 'WT'  in folder:
        WT_Cells.append(folder[:-2])
    if 'YAC128'  in folder:
        YAC128_Cells.append(folder[:-2])
        
WT_Cells = np.unique(WT_Cells)
YAC128_Cells = np.unique(YAC128_Cells)


Cell 14_YAC128_G
Cell 14_YAC128_R
Cell 17_WT_G
Cell 17_WT_R
Cell 18_WT_G
Cell 18_WT_R
Cell 22_YAC128_G
Cell 22_YAC128_R
Cell 29_YAC128_G
Cell 29_YAC128_R


# <h> Cycle Through WT Cells to Extract Peak Data

In [26]:
WT_Stats = pd.DataFrame()
WT_Peaks = pd.DataFrame()

for cell in WT_Cells:
    gcamp, rcamp = Cell2Trace(path, cell)
 
    # Calculate df/f and perform NND
    dffG = deltaFOverF0(gcamp, HZ)
    dffG = nndSmooth(dffG, HZ, tau=1)
    dffR = deltaFOverF0(rcamp, HZ)
    dffR = nndSmooth(dffR, HZ, tau=1)
    
    cell_stats, peak_data = signal_analysis(cell, dffG, dffR)
    
    WT_Stats = WT_Stats.append(cell_stats, ignore_index=True)
    WT_Peaks = WT_Peaks.append(peak_data, ignore_index=True)



NameError: name 'nonNegativeDeconvolution' is not defined

# <h2> Display Results for WT

In [ ]:
display(WT_Stats)
display(WT_Peaks)


# <h> Cycle Through YAC128 Cells to Extract Peak Data

In [ ]:
YAC128_Stats = pd.DataFrame()
YAC128_Peaks = pd.DataFrame()

for cell in YAC128_Cells:
    gcamp, rcamp = Cell2Trace(path, cell)
 
    # Calculate df/f and perform NND
    dffG = pntfilters.deltaFOverF0(gcamp, HZ)
    dffG = pntfilters.nndSmooth(dffG, HZ, tau=1)
    dffR = pntfilters.deltaFOverF0(rcamp, HZ)
    dffR = pntfilters.nndSmooth(dffR, HZ, tau=1)
    
    cell_stats, peak_data = signal_analysis(cell, dffG, dffR)
    
    YAC128_Stats = YAC128_Stats.append(cell_stats, ignore_index=True)
    YAC128_Peaks = YAC128_Peaks.append(peak_data, ignore_index=True)



# <h2> Display Results for YAC128

In [ ]:
display(YAC128_Stats)
display(YAC128_Peaks)
